In [ ]:
# import google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류
- 이미지 binary 분류 과제
- 담당: 이녕민M

## Import Libraries

In [ ]:
# !pip install opencv-python
!pip install transformers

     |████████████████████████████████| 3.5 MB 8.1 MB/s 
     |████████████████████████████████| 596 kB 78.1 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
     |████████████████████████████████| 6.8 MB 71.1 MB/s 
     |████████████████████████████████| 895 kB 57.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

from torchvision.models import efficientnet_b0, resnet18, vgg16_bn

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.utils import shuffle
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup

## Set Arguments & hyperparameters

In [ ]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [ ]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= '/content/drive/MyDrive/AIConnect/Covid infection/train'
NUM_CLS = 2

EPOCHS = 30
BATCH_SIZE = 16
LEARNING_RATE = 0.0005
EARLY_STOPPING_PATIENCE = 10
INPUT_SHAPE = 224

os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataloader

#### Train & Validation Set loader

In [ ]:
train = sorted(glob('/content/drive/MyDrive/AIConnect/Covid infection/train/train/*'))
test = sorted(glob('/content/drive/MyDrive/AIConnect/Covid infection/test/test/*'))
labelsss = pd.read_csv('/content/drive/MyDrive/AIConnect/Covid infection/train/train.csv')['COVID']

In [ ]:
import natsort

In [ ]:
train = natsort.natsorted(train)
test = natsort.natsorted(test)

In [ ]:
# k-fold cross validation

skf = StratifiedKFold(n_splits = 10)
train = shuffle(train, random_state=RANDOM_SEED)

folds = []
for index, (train_index, val_index) in enumerate(skf.split(train, labelsss)):
    folds.append((train_index, val_index))


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, files, input_shape):
        self.files = files
        self.input_shape = input_shape
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        file = self.files[i] # 파일 경로
        file_name = file.split('/')[-1] #  이것이 파일 이름. 확장자 포함
        
        # data = copy.deepcopy(self.db.loc[index]) 

        # Loading image # data 의  file_name 을 가진 image read
        
        cvimg = cv2.imread(file, cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION) 
        
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % f'{file_name}')

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, labelsss.iloc[i]

## Model

In [ ]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.resnetmodel = efficientnet_b0(pretrained=False)
        
        self.fc = nn.Sequential(nn.Linear(1000, 512), nn.ReLU(),
                                nn.Linear(512, 2),
                                nn.Sigmoid())
        
    def forward(self, x):
        x = self.resnetmodel(x)
        return self.fc(x)

model = Model()



## Utils
### EarlyStopper

In [ ]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)

### Trainer

In [ ]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

### Metrics

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [ ]:
for i in range(len(folds)):
    train_idx, val_idx = folds[i]

    # data
    train_dataset = CustomDataset(files=np.array(train)[train_idx], input_shape=INPUT_SHAPE)
    validation_dataset = CustomDataset(files=np.array(train)[val_idx], input_shape=INPUT_SHAPE)
    print('num of train: ', len(train_dataset))
    print('num of val: ', len(validation_dataset))

    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True) # drop_last=True 로 설정하면 batch로 나눈 나머지를 제거
    validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
    print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))


    # Load Model
    model = model.to(DEVICE)

    # # Save Initial Model
    # torch.save(model.state_dict(), 'initial.pt')

    # Set optimizer, scheduler, loss function, metric function
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
    loss_fn = nn.BCELoss()
    metric_fn = get_metric_fn


    # Set trainer
    trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

    # Set earlystopper
    early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)


    for epoch_index in tqdm(range(EPOCHS)):

        trainer.train_epoch(train_dataloader, epoch_index)
        trainer.validate_epoch(validation_dataloader, epoch_index)

        # early_stopping check
        early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

        if early_stopper.stop:
            print('Early stopped')
            break

        if early_stopper.save_model:
            check_point = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict()
            }
            torch.save(check_point, 'best.pt')
            torch.save(model.state_dict(), f'{i}th_model.pt')
            early_stopper.save_model = False
            # trainer
            # trainer = Trainer(model=custom_model, folder='/content'+f'/fold{i+1}', fold=i+1)


num of train:  581
num of val:  65
Train set samples: 581 Val set samples: 65


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.71500597851617, Acc: 0.4600694444444444, F1-Macro: 0.3852046332046332


  3%|▎         | 1/30 [00:07<03:46,  7.80s/it]

Epoch 0, Val loss: 0.8675224334001541, Acc: 0.5538461538461539, F1-Macro: 0.3564356435643565
Epoch 1, Train loss: 0.7148210099765233, Acc: 0.4756944444444444, F1-Macro: 0.4631641319375864
Epoch 1, Val loss: 0.8614842742681503, Acc: 0.4461538461538462, F1-Macro: 0.30851063829787234
Validation loss decreased 0.8675224334001541 -> 0.8614842742681503


...

Epoch 9, Val loss: 4.958037614822388, Acc: 0.53125, F1-Macro: 0.5294117647058825
Early stopping counter 8/10
Epoch 10, Train loss: 0.058063859618934135, Acc: 0.9791666666666666, F1-Macro: 0.9790755966724387


 37%|███▋      | 11/30 [01:25<02:28,  7.79s/it]

Epoch 10, Val loss: 6.186257521311442, Acc: 0.53125, F1-Macro: 0.5294117647058825
Early stopping counter 9/10
Epoch 11, Train loss: 0.049162943454991496, Acc: 0.9756944444444444, F1-Macro: 0.975599128540305


 37%|███▋      | 11/30 [01:33<02:41,  8.51s/it]

Epoch 11, Val loss: 4.770620584487915, Acc: 0.53125, F1-Macro: 0.5294117647058825
Early stopping counter 10/10
Early stopped


#### Load model and other utils

### epoch 단위 학습 진행

## Inference
### 모델 로드

In [ ]:
TRAINED_MODEL_PATH = 'best.pt'
model1_path = '/content/0th_model.pt'
model3_path = '/content/2th_model.pt'
model4_path = '/content/3th_model.pt'
model5_path = '/content/4th_model.pt'
model6_path = '/content/5th_model.pt'
model7_path = '/content/6th_model.pt'
model8_path = '/content/7th_model.pt'
model9_path = '/content/8th_model.pt'
model10_path ='/content/9th_model.pt'

In [ ]:
model1= Model()
model1.load_state_dict(torch.load(model1_path))
model3= Model()
model3.load_state_dict(torch.load(model3_path))
model4= Model()
model4.load_state_dict(torch.load(model4_path))
model5= Model()
model5.load_state_dict(torch.load(model5_path))
model6= Model()
model6.load_state_dict(torch.load(model6_path))
model7= Model()
model7.load_state_dict(torch.load(model7_path))
model8= Model()
model8.load_state_dict(torch.load(model8_path))
model9= Model()
model9.load_state_dict(torch.load(model9_path))
model10= Model()
model10.load_state_dict(torch.load(model10_path))


<All keys matched successfully>

AttributeError: ignored

### Load dataset

In [ ]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [ ]:
# Load dataset & dataloader
test_DATA_DIR = '/content/drive/MyDrive/AIConnect/Covid infection/test/'
test_dataset = TestDataset(data_dir=test_DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


In [ ]:

def predict(models, loader):
    model1, model3, model4, model5 = models

    preds = []
    for bi, data in enumerate(tqdm(loader)):
        data = [x for x in data]
        
        output1 = nn.Softmax(dim=-1)(model1(img))

        output3 = nn.Softmax(dim=-1)(model3(img))
        output4 = nn.Softmax(dim=-1)(model4(img))
        output5 = nn.Softmax(dim=-1)(model5(img))

        output = output1 + output3 + output4 + output5
        pred = torch.argmax(output, dim=1).cpu().tolist()
        preds.extend(pred)
    return preds
  

# inference
models = [model1, model3, model4, model5]

test_dataset = TestDataset(data_dir=test_DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

with torch.no_grad():
    preds = predict(models, test_dataloader)

Loading test dataset..


100%|██████████| 7/7 [00:05<00:00,  1.25it/s]


### 추론 진행

In [ ]:
model3= Model().to(DEVICE)
model3.load_state_dict(torch.load(model3_path))
# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

1it [00:00,  1.75it/s]

tensor([[0.3866, 0.0935],
        [0.5200, 0.8352],
        [0.5100, 0.7865],
        [0.5329, 0.9217],
        [0.4096, 0.1771],
        [0.5071, 0.7504],
        [0.5199, 0.8215],
        [0.5074, 0.7625],
        [0.4434, 0.1833],
        [0.5221, 0.8579],
        [0.4038, 0.0732],
        [0.5339, 0.9239],
        [0.4897, 0.5963],
        [0.5307, 0.9248],
        [0.5277, 0.9013],
        [0.5209, 0.8396]])


2it [00:01,  1.78it/s]

tensor([[0.4643, 0.3841],
        [0.3965, 0.1184],
        [0.4709, 0.4532],
        [0.3798, 0.0803],
        [0.4320, 0.1741],
        [0.4858, 0.5780],
        [0.4295, 0.1819],
        [0.2969, 0.0608],
        [0.4979, 0.6672],
        [0.4974, 0.6649],
        [0.5069, 0.7766],
        [0.5219, 0.8970],
        [0.5464, 0.9815],
        [0.5296, 0.9044],
        [0.5438, 0.9463],
        [0.5268, 0.8806]])


3it [00:01,  1.83it/s]

...

tensor([[0.4744, 0.4297],
        [0.4150, 0.0994],
        [0.4151, 0.1577],
        [0.5370, 0.9234],
        [0.5214, 0.8395],
        [0.4695, 0.3621],
        [0.4804, 0.5079],
        [0.5379, 0.9396],
        [0.4781, 0.4716],
        [0.5113, 0.7779],
        [0.3563, 0.0435],
        [0.2227, 0.0167],
        [0.5362, 0.9270],
        [0.5107, 0.7610],
        [0.5405, 0.9428],
        [0.4286, 0.1579]])


5it [00:02,  1.84it/s]

tensor([[0.4699, 0.3589],
        [0.5425, 0.9371],
        [0.5485, 0.9570],
        [0.5335, 0.9655],
        [0.4653, 0.3724],
        [0.2431, 0.0275],
        [0.5078, 0.7485],
        [0.1485, 0.0049],
        [0.5037, 0.7281],
        [0.5475, 0.9682],
        [0.4937, 0.6106],
        [0.4374, 0.1416],
        [0.3691, 0.1027],
        [0.5123, 0.8239],
        [0.5396, 0.9437],
        [0.5581, 0.9989]])


7it [00:03,  2.03it/s]

tensor([[0.4892, 0.5592],
        [0.4909, 0.5915],
        [0.4047, 0.1196],
        [0.5229, 0.8393],
        [0.3259, 0.0515],
        [0.4442, 0.2253],
        [0.4473, 0.2473],
        [0.4960, 0.6675],
        [0.4564, 0.2409],
        [0.4683, 0.3661],
        [0.5372, 0.9198],
        [0.4620, 0.3125],
        [0.5036, 0.7603],
        [0.3879, 0.0959],
        [0.5357, 0.9109],
        [0.3979, 0.0884]])
tensor([[0.5351, 0.9250],
        [0.4744, 0.4087],
        [0.5043, 0.7277],
        [0.5026, 0.7141]])


### 결과 저장

In [ ]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('prediction3333.csv', index=False)